In [1]:
# --- Notebook banner (reproducible skeleton) ---
import os, sys, math, time, random, json, textwrap, warnings
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from pathlib import Path

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({'font.size': 14, 'figure.figsize': (10, 6), 'figure.dpi': 150})
np.set_printoptions(suppress=True, linewidth=120, precision=4)

# --- Custom plotting and noting functions ---
def note(msg, **kwargs): print(f"📝 {textwrap.fill(msg, width=100)}", **kwargs)
def sec(title): print(f"\n{100*'='}\n| {title.upper()} |\n{100*'='}")

NOTEBOOK_PATH = Path('9.2_Autograding_with_Otter.ipynb').resolve()
IMAGE_DIR = NOTEBOOK_PATH.parent / 'images' / NOTEBOOK_PATH.stem
IMAGE_DIR.mkdir(parents=True, exist_ok=True)

note(f"Environment initialized.")

📝 Environment initialized.


# Part 9: Scientific Communication and Pedagogy
## Chapter 9.2: Scalable Assessment with Otter-Grader

### Introduction: The Pedagogical Case for Autograding

A primary challenge in teaching computational courses is providing timely, consistent, and high-quality feedback. Manual grading is slow, prone to bias, and logistically overwhelming, especially in large courses. **Otter-Grader** is a powerful open-source framework designed to solve this problem by automating the entire lifecycle of a computational assignment.

Otter transforms grading into a pedagogical tool that enables:
- **Immediate Formative Feedback:** Students can check their work against public tests as they go.
- **Consistency and Fairness:** All submissions are evaluated against the same tests in the same environment.
- **Scalability:** It allows a small instructional team to support hundreds or thousands of students.
- **Richer Assignments:** By automating routine checks, instructors can focus on designing more creative problems and providing feedback on open-ended questions.

### Learning Objectives

By the end of this chapter, you will master a complete, modern workflow for computational assignments. You will be able to:

*   **Understand the Otter Workflow:** Outline the four main stages: authoring, generating, submission, and grading.
*   **Author Master Notebooks:** Use Otter's comment-based syntax to create a master notebook with questions, solutions, public tests, and hidden tests.
*   **Handle Different Question Types:** Create questions for code, written responses (manual grading), and plots.
*   **Generate Student Materials:** Use the `otter assign` command to generate student-facing notebooks and the complete autograder configuration.
*   **Write Robust Tests:** Move beyond simple assertions by testing for edge cases and conceptual correctness.

## 1. The Otter-Grader Workflow

The Otter workflow is a four-stage process that separates the instructor's, student's, and grader's roles.

1.  **Authoring (The `master` Notebook):** The instructor creates a single "master" notebook containing everything: questions, complete solutions, public tests (visible to students), and hidden tests (for final grading).

2.  **Generation (`otter assign`):** The instructor runs `otter assign` on the master notebook. This command generates:
    *   A **student version** of the notebook, with solutions and hidden tests removed.
    *   An **autograder configuration** (`autograder.zip`), which contains everything needed for grading.

3.  **Submission (`otter.export`):** Students complete the assignment and run `otter.export()` in their notebook to generate a `.zip` file for submission.

4.  **Grading (`otter grade`):** The instructor runs `otter grade`, which executes each student's submission in a secure, isolated Docker container, runs all tests, and produces a final grade sheet.

## 2. Code Lab: Authoring a Master Assignment Notebook

Let's create a complete master notebook for a small assignment. This assignment will have three questions: one autograded code question, one manually graded plot, and one manually graded free-response question.

### Step 1: Create the Master Notebook File
First, create a new notebook file named `master.ipynb`.

### Step 2: Add an Initialization Cell
The first cell should import necessary libraries and the `otter` helper functions.

```json
{
 "cell_type": "code",
 "metadata": {},
 "source": [
  "import pandas as pd\n",
  "import numpy as np\n",
  "import matplotlib.pyplot as plt\n",
  "import otter\n",
  "grader = otter.Notebook(\"assignment1.ipynb\")"
 ]
}
```

### Step 3: Add Question 1 (Autograded Code)
This question asks for a function. We provide the prompt, the full solution, and both public and hidden tests within a single cell, using Otter's comment-based tags.

```json
{
 "cell_type": "markdown",
 "source": ["## Question 1: Fibonacci Sequence"]
},
{
 "cell_type": "code",
 "source": [
  "# BEGIN QUESTION\n",
  "# NAME: q1_fibonacci\n",
  "# POINTS: 2\n",
  "\n",
  "# --- BEGIN SOLUTION ---\n",
  "def fibonacci(n):\n",
  "    if n <= 0: return []\n",
  "    if n == 1: return [0]\n",
  "    sequence = [0, 1]\n",
  "    while len(sequence) < n:\n",
  "        next_val = sequence[-1] + sequence[-2]\n",
  "        sequence.append(next_val)\n",
  "    return sequence[:n] # Ensure correct length\n",
  "# --- END SOLUTION ---\n",
  "\n",
  "# --- BEGIN PROMPT\n",
  "def fibonacci(n):\n",
  "    \"\"\"Generates the first n numbers of the Fibonacci sequence.\"\"\"\n",
  "    ...\n",
  "# --- END PROMPT\n",
  "\n",
  "# Public tests\n",
  "assert fibonacci(5) == [0, 1, 1, 2, 3]\n",
  "assert fibonacci(8) == [0, 1, 1, 2, 3, 5, 8, 13]\n",
  "\n",
  "# Hidden tests\n",
  "# --- BEGIN HIDDEN TESTS ---\n",
  "assert fibonacci(0) == []\n",
  "assert fibonacci(1) == [0]\n",
  "assert fibonacci(2) == [0, 1]\n",
  "# --- END HIDDEN TESTS ---\
",
  "# END QUESTION"
 ],
 "metadata": {}
}
```

### Step 4: Add Question 2 (Manually Graded Plot)
This question requires a human to assess the correctness of a plot.

```json
{
 "cell_type": "markdown",
 "source": ["## Question 2: Plotting"]
},
{
 "cell_type": "code",
 "source": [
  "# BEGIN QUESTION\n",
  "# NAME: q2_plot\n",
  "# POINTS: 1\n",
  "# MANUAL: true\n",
  "\n",
  "# --- BEGIN SOLUTION ---\n",
  "fib_15 = fibonacci(15)\n",
  "plt.plot(range(15), fib_15, marker='o')\n",
  "plt.title('First 15 Fibonacci Numbers')\n",
  "plt.xlabel('Index')\n",
  "plt.ylabel('Fibonacci Number')\n",
  "plt.grid(True)\n",
  "plt.show()\n",
  "# --- END SOLUTION ---\n",
  "# END QUESTION"
 ],
 "metadata": {}
}
```

### Step 5: Add Question 3 (Manually Graded Text)
This question asks for a written interpretation.

```json
{
 "cell_type": "markdown",
 "source": [
  "<!-- BEGIN QUESTION -->\n",
  "<!-- NAME: q3_interpretation -->\n",
  "<!-- POINTS: 1 -->\n",
  "<!-- MANUAL: true -->\n",
  "\n",
  "Describe the growth pattern of the Fibonacci sequence based on your plot.\n",
  "\n",
  "<!-- **SOLUTION**:\n",
  "The Fibonacci sequence exhibits exponential growth. The numbers increase at an accelerating rate, which is clearly visible in the upward-curving plot. Each number is the sum of the two preceding ones, leading to a growth factor that approaches the Golden Ratio (approximately 1.618). -->\n",
  "\n",
  "<!-- END QUESTION -->"
 ]
}
```

### Step 6: Generate the Assignment
With `master.ipynb` complete, you run the `otter assign` command in your terminal:
```bash
otter assign master.ipynb dist
```
This creates a `dist` directory containing:
- `autograder/`: The student-facing version of the notebook.
- `autograder.zip`: The complete grading configuration for the autograder.

## 3. Writing Robust Tests

The quality of an autograded assignment depends on the quality of its tests. Hidden tests should go beyond the basic cases and check for **edge cases** and **invariants**.

### 3.1. Testing Edge Cases
For our `fibonacci` function, good edge cases to test include:
- `fibonacci(0)`: Should return an empty list.
- `fibonacci(1)`: Should return `[0]`.
- `fibonacci(2)`: Should return `[0, 1]`.

### 3.2. Property-Based Testing with Hypothesis
A more powerful technique is **property-based testing**, which checks if a function satisfies general properties for a wide range of automatically generated inputs. The `hypothesis` library is the standard for this in Python.

A key property of our `fibonacci` function is that for any `n > 2`, `fibonacci(n)[-1]` must equal `fibonacci(n)[-2] + fibonacci(n)[-3]`. We can write a test for this.

In [2]:
sec("Property-Based Test Demonstration")

try:
    from hypothesis import given, strategies as st, settings, HealthCheck
    HYPOTHESIS_AVAILABLE = True
except ImportError:
    HYPOTHESIS_AVAILABLE = False

if HYPOTHESIS_AVAILABLE:
    # This is the instructor's reference solution for the test
    def fibonacci(n):
        if n <= 0: return []
        if n == 1: return [0]
        sequence = [0, 1]
        while len(sequence) < n:
            next_val = sequence[-1] + sequence[-2]
            sequence.append(next_val)
        return sequence[:n]

    # Define a Hypothesis strategy: generate integers from 3 to 100
    integer_strategy = st.integers(min_value=3, max_value=100)

    # The @given decorator runs this test with many examples from the strategy
    @settings(deadline=None, suppress_health_check=[HealthCheck.too_slow])
    @given(n=integer_strategy)
    def test_fibonacci_property(n):
        # In a real test, this would call the student's submitted function
        seq = fibonacci(n)
        # Property: Each element is the sum of the two preceding ones
        assert seq[-1] == seq[-2] + seq[-3]

    note("Running property-based test for the Fibonacci function...")
    test_fibonacci_property()
    note("Property-based tests passed successfully.")
else:
    note("Hypothesis not installed. Skipping property-based test demonstration.")


| PROPERTY-BASED TEST DEMONSTRATION |
📝 Hypothesis not installed. Skipping property-based test demonstration.


## 4. Exercises

1.  **Install Otter-Grader:** Follow the instructions on the [Otter-Grader documentation](https://otter-grader.readthedocs.io/en/latest/installation.html) to install the `otter-grader` package (`pip install otter-grader`).

2.  **Create the Master Notebook:** Create a new Jupyter notebook named `master.ipynb`. Following the examples in Section 2, add the initialization cell and the three questions (Fibonacci, plot, and interpretation) to the notebook.

3.  **Generate the Assignment:** From your terminal, run `otter assign master.ipynb ./assignment1_dist`. This will create a directory `assignment1_dist`.

4.  **Inspect the Output:** Open the student notebook located at `assignment1_dist/student/master.ipynb`. Verify that the solution code blocks have been replaced with the prompt and that the hidden tests and markdown solutions are gone.

5.  **Add a New Test:** Add a new hidden test to `q1_fibonacci` in your `master.ipynb` that checks if the function works for a large number, e.g., `assert len(fibonacci(100)) == 100`. Rerun `otter assign` and confirm the student version remains unchanged.